In [1]:
import pandas as pd
import datetime
import os
import matplotlib.pyplot as plt
import requests
import numpy as np

In [29]:
def get_osm_map_image(latitude, longitude, zoom=15, width=600, height=400):
    base_url = "https://www.openstreetmap.org/export/embed"
    
    # Set parameters for the API request
    params = {
        'bbox': f'{longitude-0.005},{latitude-0.005},{longitude+0.005},{latitude+0.005}',
        'layer': 'mapnik',
        'marker': f'{latitude},{longitude}',
        'zoom': zoom,
        'width': width,
        'height': height
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # Save the map image to a file
        with open("osm_map_image.html", "w") as file:
            file.write(response.text)
        
        
    else:
        print(f"Failed to retrieve map image. Status code: {response.status_code}")

In [39]:
df = pd.read_excel('HEDNO_KPIs_4_2024.xlsx', engine='openpyxl')
df.head()

,Installation,Transformer,Distribution,Total energy consumption (MWh),Nr. of power alarms L1,Min duration of power alarms L1 (min),Max duration of power alarms L1 (min),Nr. of power alarms L2,Min duration of power alarms L2 (min),Max duration of power alarms L2 (min),...,% of time of Current unbalance alarms,Nr. of Power Fails (outage),Min time of Power Fails (sec),Max time of Power Fails (sec),Nr. of Voltage dips,Avg time of Voltage dips (msec),Nr. of Voltage swells,Avg time of Voltage swells (msec),Latitude,Longitude
0,Ρ 3-19 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-423,NaN,12.00,318.0,1.0,112.0,457.0,1.0,177.0,...,0.0,0,0.0,0.0,0,0.0,0,0,37.889600,23.915061
1,Ρ 3-19 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-423,102.408.000766,1.37,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0.0,0.0,0,0.0,0,0,37.889600,23.915061
2,Ρ 3-19 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-423,102.408.000767,0.02,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0,0.0,0,0,37.889600,23.915061
3,ΜΠ-161 ΠΑΛΛΗΝΗΣ,ΜΠ-297,NaN,118.35,0.0,NaN,NaN,0.0,NaN,NaN,...,0.0,0,0.0,0.0,2,357.0,0,0,37.969158,23.853149
4,ΜΠ-22 Κ/Δ ΠΑΛΛΗΝΗΣ,ΜΠ-013,NaN,145.48,0.0,NaN,NaN,0.0,NaN,NaN,...,0.0,0,0.0,0.0,1,314.0,0,0,37.958051,23.852520


In [40]:
###### TEST to get new columns

grouped = df.groupby('Transformer')
for i, group in grouped:
    
    group['Nr. of feeder overcurrent alarms'] = group['Nr. of overcurrent alarms L1']+group['Nr. of overcurrent alarms L2']+group['Nr. of overcurrent alarms L3']
    group['Min duration of feeder overcurrent alarms (minutes)'] = group[['Min duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)']].min(axis=1)
    group['Max duration of feeder overcurrent alarms (minutes)'] = group[['Max duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L3 (min)']].max(axis=1)
    #group['Min duration of feeder overcurrent alarms'] = np.min(group['Min duration of overcurrent alarms L1 (min)'],group['Min duration of overcurrent alarms L2 (min)'],group['Min duration of overcurrent alarms L3 (min)'])
    #group['Max duration of feeder overcurrent alarms'] = np.max(group['Max duration of overcurrent alarms L1 (min)'],group['Max duration of overcurrent alarms L2 (min)'],group['Max duration of overcurrent alarms L3 (min)'])
    
    group['Nr. of feeder overcurrent alarms'] = group['Nr. of feeder overcurrent alarms'].sum()
    group['Min duration of feeder overcurrent alarms (minutes)'] = group['Min duration of feeder overcurrent alarms (minutes)'].min()
    group['Max duration of feeder overcurrent alarms (minutes)'] = group['Max duration of feeder overcurrent alarms (minutes)'].max()

    group['Nr. of power alarms'] = group['Nr. of power alarms L1']+group['Nr. of power alarms L2']+group['Nr. of power alarms L3']
    group['Min duration of power alarms (minutes)'] = group[['Min duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Min duration of power alarms L3 (min)']].min(axis=1)
    group['Max duration of power alarms (minutes)'] = group[['Max duration of power alarms L1 (min)','Max duration of power alarms L2 (min)','Max duration of power alarms L3 (min)']].max(axis=1)
    
    #group['Min duration of power alarms'] = np.min(group['Min duration of power alarms L1 (min)'],group['Min duration of power alarms L2 (min)'],group['Min duration of power alarms L3 (min)'])
    #group['Max duration of power alarms'] = np.max(group['Max duration of power alarms L1 (min)'],group['Max duration of power alarms L2 (min)'],group['Max duration of power alarms L3 (min)'])

    group = group.drop(['Nr. of overcurrent alarms L1','Nr. of overcurrent alarms L2','Nr. of overcurrent alarms L3','Min duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)','Max duration of overcurrent alarms L3 (min)'],axis=1)
    group = group.drop(['Nr. of power alarms L1','Nr. of power alarms L2','Nr. of power alarms L3','Min duration of power alarms L1 (min)','Max duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Max duration of power alarms L2 (min)','Min duration of power alarms L3 (min)','Max duration of power alarms L3 (min)'],axis=1)
    group = pd.DataFrame(group.iloc[0]).transpose()
    columns = group.columns 
    print(columns)

Index(['Installation', 'Transformer', 'Distribution',
       'Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Latitude', 'Longitude', 'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')
Index(['Installation', 'Transformer', 'Distribution',
       'Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage

In [41]:
summary = pd.DataFrame(columns=columns)


grouped = df.groupby('Transformer')
for i, group in grouped:
    group['Nr. of feeder overcurrent alarms'] = group['Nr. of overcurrent alarms L1']+group['Nr. of overcurrent alarms L2']+group['Nr. of overcurrent alarms L3']
    group['Min duration of feeder overcurrent alarms (minutes)'] = group[['Min duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)']].min(axis=1)
    group['Max duration of feeder overcurrent alarms (minutes)'] = group[['Max duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L3 (min)']].max(axis=1)
    #group['Min duration of feeder overcurrent alarms'] = np.min(group['Min duration of overcurrent alarms L1 (min)'],group['Min duration of overcurrent alarms L2 (min)'],group['Min duration of overcurrent alarms L3 (min)'])
    #group['Max duration of feeder overcurrent alarms'] = np.max(group['Max duration of overcurrent alarms L1 (min)'],group['Max duration of overcurrent alarms L2 (min)'],group['Max duration of overcurrent alarms L3 (min)'])
    
    group['Nr. of feeder overcurrent alarms'] = group['Nr. of feeder overcurrent alarms'].sum()
    group['Min duration of feeder overcurrent alarms (minutes)'] = group['Min duration of feeder overcurrent alarms (minutes)'].min()
    group['Max duration of feeder overcurrent alarms (minutes)'] = group['Max duration of feeder overcurrent alarms (minutes)'].max()

    group['Nr. of power alarms'] = group['Nr. of power alarms L1']+group['Nr. of power alarms L2']+group['Nr. of power alarms L3']
    group['Min duration of power alarms (minutes)'] = group[['Min duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Min duration of power alarms L3 (min)']].min(axis=1)
    group['Max duration of power alarms (minutes)'] = group[['Max duration of power alarms L1 (min)','Max duration of power alarms L2 (min)','Max duration of power alarms L3 (min)']].max(axis=1)
    
    #group['Min duration of power alarms'] = np.min(group['Min duration of power alarms L1 (min)'],group['Min duration of power alarms L2 (min)'],group['Min duration of power alarms L3 (min)'])
    #group['Max duration of power alarms'] = np.max(group['Max duration of power alarms L1 (min)'],group['Max duration of power alarms L2 (min)'],group['Max duration of power alarms L3 (min)'])

    group = group.drop(['Nr. of overcurrent alarms L1','Nr. of overcurrent alarms L2','Nr. of overcurrent alarms L3','Min duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)','Max duration of overcurrent alarms L3 (min)'],axis=1)
    group = group.drop(['Nr. of power alarms L1','Nr. of power alarms L2','Nr. of power alarms L3','Min duration of power alarms L1 (min)','Max duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Max duration of power alarms L2 (min)','Min duration of power alarms L3 (min)','Max duration of power alarms L3 (min)'],axis=1)
    group = pd.DataFrame(group.iloc[0]).transpose()

    summary = summary.append(group, ignore_index=True)

C:\Users\EmilyP\AppData\Local\Temp\ipykernel_4280\673294417.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_4280\673294417.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_4280\673294417.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_4280\673294417.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_

In [42]:
# summary = pd.DataFrame(columns=columns)


# grouped = df.groupby('Transformer')
# for i, group in grouped:
    
#     group['Nr. of feeder overcurrent alarms'] = group['Nr. of overcurrent alarms L1']+group['Nr. of overcurrent alarms L2']+group['Nr. of overcurrent alarms L3']
#     group['% of time of feeder overcurrent alarms'] = group['% of time of overcurrent alarms L1']+group['% of time of overcurrent alarms L2']+group['% of time of overcurrent alarms L3']
#     group['Nr. of feeder overcurrent alarms'] = group['Nr. of feeder overcurrent alarms'].sum()
#     group['% of time of feeder overcurrent alarms'] = group['% of time of feeder overcurrent alarms'].sum()

#     group['Nr. of power alarms'] = group['Nr. of power alarms L1']+group['Nr. of power alarms L2']+group['Nr. of power alarms L3']
#     group['% of time of power alarms'] = group['% of time of power alarms L1']+group['% of time of power alarms L2']+group['% of time of power alarms L3']

#     group = group.drop(['Nr. of overcurrent alarms L1','Nr. of overcurrent alarms L2','Nr. of overcurrent alarms L3','% of time of overcurrent alarms L1','% of time of overcurrent alarms L2','% of time of overcurrent alarms L3'],axis=1)
#     group = group.drop(['Nr. of power alarms L1','Nr. of power alarms L2','Nr. of power alarms L3','% of time of power alarms L1','% of time of power alarms L2','% of time of power alarms L3'],axis=1)
#     group = pd.DataFrame(group.iloc[0]).transpose()
#     # columns = group.columns 

#     summary = summary.append(group, ignore_index=True)
#     #print(group.columns)


In [43]:
summary.head()

,Installation,Transformer,Distribution,Total energy consumption (MWh),Nr. of Voltage unbalance alarms,% of time of Voltage unbalance alarms,Nr. of Current unbalance alarms,% of time of Current unbalance alarms,Nr. of Power Fails (outage),Min time of Power Fails (sec),...,Nr. of Voltage swells,Avg time of Voltage swells (msec),Latitude,Longitude,Nr. of feeder overcurrent alarms,Min duration of feeder overcurrent alarms (minutes),Max duration of feeder overcurrent alarms (minutes),Nr. of power alarms,Min duration of power alarms (minutes),Max duration of power alarms (minutes)
0,1-15 ΜΑΡΚΟΠΟΥΛΟΥ,MM-185,NaN,98.99,0.0,0.0,0,0.0,1,0.000221,...,1,166,37.94104,24.002002,0.0,NaN,NaN,0.0,NaN,NaN
1,Ρ-380 ΣΠΑΤΩΝ,MM-359,NaN,14.02,0.0,0.0,0,NaN,0,0.0,...,0,0,37.93041,23.875718,0.0,NaN,NaN,0.0,NaN,NaN
2,Ρ-230 ΝΕΑΣ ΜΑΚΡΗΣ,ΜΓ-019,NaN,17.48,0.0,0.0,0,NaN,1,0.000437,...,0,0,38.200179,23.99773,0.0,NaN,NaN,0.0,NaN,NaN
3,Ρ-280 ΛΑΥΡΙΟΥ,ΜΛ-034,NaN,87.51,0.0,0.0,0,NaN,0,0.0,...,0,0,37.748704,23.906775,0.0,NaN,NaN,0.0,NaN,NaN
4,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-014,NaN,14.58,0.0,0.0,0,NaN,0,0.0,...,0,0,37.862998,23.85289,0.0,NaN,NaN,0.0,NaN,NaN


In [44]:
summary.loc[summary['Avg time of Voltage dips (msec)']<0,'Avg time of Voltage dips (msec)'] = 'Unknown'

In [45]:
summaryCols = summary.drop(['Installation','Transformer','Distribution','Latitude','Longitude'],axis=1).columns
print(summaryCols)

Index(['Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')


In [46]:
# summary.rename(columns={'Total energy consumption (KWh)':'Total energy consumption (MWh)'},inplace=True)
# # summaryCols = summary.columns
# summaryCols


In [47]:
summary.to_excel('HEDNO_KPIs_simplified_4_2024.xlsx', index=False)

In [48]:
summary = summary.fillna(0)
summary

,Installation,Transformer,Distribution,Total energy consumption (MWh),Nr. of Voltage unbalance alarms,% of time of Voltage unbalance alarms,Nr. of Current unbalance alarms,% of time of Current unbalance alarms,Nr. of Power Fails (outage),Min time of Power Fails (sec),...,Nr. of Voltage swells,Avg time of Voltage swells (msec),Latitude,Longitude,Nr. of feeder overcurrent alarms,Min duration of feeder overcurrent alarms (minutes),Max duration of feeder overcurrent alarms (minutes),Nr. of power alarms,Min duration of power alarms (minutes),Max duration of power alarms (minutes)
0,1-15 ΜΑΡΚΟΠΟΥΛΟΥ,MM-185,0,98.99,0.0,0.00,0,0.00,1,2.206944e-04,...,1,166,37.941040,24.002002,0.0,0,0,0.0,0.0,0.0
1,Ρ-380 ΣΠΑΤΩΝ,MM-359,0,14.02,0.0,0.00,0,0.00,0,0.000000e+00,...,0,0,37.930410,23.875718,0.0,0,0,0.0,0.0,0.0
2,Ρ-230 ΝΕΑΣ ΜΑΚΡΗΣ,ΜΓ-019,0,17.48,0.0,0.00,0,0.00,1,4.370255e-04,...,0,0,38.200179,23.997730,0.0,0,0,0.0,0.0,0.0
3,Ρ-280 ΛΑΥΡΙΟΥ,ΜΛ-034,0,87.51,0.0,0.00,0,0.00,0,0.000000e+00,...,0,0,37.748704,23.906775,0.0,0,0,0.0,0.0,0.0
4,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-014,0,14.58,0.0,0.00,0,0.00,0,0.000000e+00,...,0,0,37.862998,23.852890,0.0,0,0,0.0,0.0,0.0
5,1-15 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-062,0,40.40,0.0,0.00,0,0.00,2,1.273148e-07,...,0,0,37.889789,23.920611,0.0,0,0,0.0,0.0,0.0
6,Ρ-410 ΣΠΑΤΩΝ,ΜΜ-092,0,19.85,0.0,0.00,0,0.00,3,3.518519e-06,...,0,0,37.934058,23.880341,0.0,0,0,0.0,0.0,0.0
7,3-15 ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-141,0,5.08,0.0,0.00,0,0.00,0,0.000000e+00,...,0,0,37.905610,23.980450,0.0,0,0,0.0,0.0,0.0
8,3-22 Κ/Δ ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-197,0,41.14,0.0,0.00,0,0.00,2,1.562500e-06,...,0,0,37.862493,24.034781,0.0,0,0,0.0,0.0,0.0
9,3-22 Κ/Δ ΜΑΡΚΟΠΟΥΛΟΥ,ΜΜ-218,0,20.81,0.0,0.00,0,0.00,1,2.288889e-04,...,0,0,37.867379,24.037221,0.0,0,0,0.0,0.0,0.0


In [49]:
import folium

def get_marker_color(value):
    if value > 0:
        return 'red'
    else:
        return 'blue'

m = folium.Map(location=[37.98392635351863, 23.727349678741202], tiles="OpenStreetMap", zoom_start=12)



# Add points to the map
for index, row in summary.iterrows():
    
    # tmp = pd.DataFrame(row[['Total energy consumption (KWh)','Nr. of Voltage unbalance alarms','% of time of Voltage unbalance alarms']])#.transpose()
    tmp = pd.DataFrame(row[summaryCols]).transpose()
    # tmp = tmp.reset_index(drop=True)
    # col = tmp.columns[0]
    newcol = row['Installation']+':'+row['Transformer']
    # tmp.rename(columns = {col:newcol},inplace=True)
    
    tmp = tmp.rename(index={tmp.index[0]: newcol})
    

    html = tmp.to_html(
    classes="table table-striped table-hover table-condensed table-responsive"
)
    popup = folium.Popup(html)
    # popup = row['Installation']+'\n'+row['Transformer']+'\n'
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup, icon=folium.Icon(color=get_marker_color(row['Nr. of power alarms']))).add_to(m)


m

In [50]:
m.save('HEDNO_KPIs_map_4_2024.html')
